# Demo for uproot

In [1]:
import numpy as np
import uproot
import awkward as ak

num_events = 100 #must be match to the existed root files name
data_path = "/Users/yianchen/CMS_Open_Data_Workspace/CMSSW_7_6_7/src/QCD_Jet_Fatjet/Analyzer/root_files/"
channels = [
    "ZprimeToZhToZinvhbb",
    "ZprimeToZhToZlephbb",
    "QCD_HT1500to2000",
    "QCD_HT2000toInf",
]

def get_events(idx):
    print(channels[idx])
    channel = channels[idx]
    root_file = f"{data_path}{channel}_{num_events}.root"
    events = uproot.open(root_file+f":jets/Events")
    return events

### Cut by Pt

In [2]:
events = get_events(1)
cut_events = events.arrays(
    expressions = ["fatjet_pt", "fatjet_eta", "fatjet_phi"],
    cut = "(fatjet_pt >= 800) & (fatjet_pt <= 1200)"
)

max_pt_idx = ak.firsts(ak.argsort(cut_events["fatjet_pt"], ascending=False))
max_pt_idx = max_pt_idx[ak.is_none(max_pt_idx) == False]

cut_events = cut_events[max_pt_idx]
cut_events

ZprimeToZhToZlephbb


<Array [{fatjet_pt: [1.17e+03, ... 1.18]}] type='22 * ?{"fatjet_pt": var * float...'>

### Select the jet/fatjet with max Pt

https://github.com/scikit-hep/awkward/discussions/1906#discussioncomment-4207964

In [3]:
# demo advanced index

x1 = ak.Array([
    [1,2,3],
    [4,5,6,7],
    [8,9],
])

x2 = ak.Array([
    [[-1,2], [3,4], [5,6]],
    [[1,1], [2,2], [3,3], [4,4]],
    [[1,2,3], [4,5,6]],
])

ix = ak.Array([
    [2],
    [0],
    [1]
])

np.abs(x2)

<Array [[[1, 2], [3, 4], ... 2, 3], [4, 5, 6]]] type='3 * var * var * int64'>

In [4]:
events = get_events(3)
jet_type = "fatjet"
events = events.arrays(filter_name=f"{jet_type}*")
max_index = ak.firsts(ak.argsort(events[f"{jet_type}_pt"], ascending=False))
max_index = ak.unflatten(max_index, counts=ak.ones_like(max_index))
events = events[max_index]
for i in range(5):
    print(events["fatjet_pt"][i], events["fatjet_daughter_pdgid"][i])

QCD_HT2000toInf
[1.19e+03] [... -211, 211, 22, -211, 211, 22, 22, 211, 211, 211, -211, 22, 211, -211, -211]]
[913] [... 22, 22, 22, 211, 22, -211, 22, 22, 22, -211, -211, 211, 22, 22, 22, 211]]
[906] [... 22, 211, 211, 211, 22, 22, -211, 22, -211, 22, 22, 22, 22, 22, 211, 22, -211]]
[1.54e+03] [... 211, 130, -211, 130, 130, 22, 22, -211, -211, 22, 211, 22, 22, -211, 211]]
[1.32e+03] [... 211, 130, 211, 211, 211, 130, 22, 211, 130, 22, 211, 22, 22, -211, 22, 22]]
